# Comparison


In [71]:
import pandas as pd
import numpy as np
import os

from statsmodels.tools.eval_measures import mse, meanabs, medianabs

pd.set_option('display.max_columns', None)

In [51]:
from scripts.params import get_params

params = get_params()

In [48]:
dataroute=os.path.join("..",  "data")
dumproute=os.path.join("..",  "dump")
resultsroute=os.path.join("..",  "results")

In [49]:
all_residuals = {}

for filename in os.listdir(resultsroute):
    file_path = os.path.join(resultsroute, filename)
    if os.path.isfile(file_path) and 'residual' in filename:
        all_residuals[filename] = file_path

print(all_residuals)

{'GARCH_AR_^MERV_aic_residuals.pickle': '..\\results\\GARCH_AR_^MERV_aic_residuals.pickle', 'GARCH_AR_^MERV_bic_residuals.pickle': '..\\results\\GARCH_AR_^MERV_bic_residuals.pickle', 'HMM_univ_AR_^MERV_aic_residuals.pickle': '..\\results\\HMM_univ_AR_^MERV_aic_residuals.pickle', 'HMM_univ_AR_^MERV_bic_residuals.pickle': '..\\results\\HMM_univ_AR_^MERV_bic_residuals.pickle', 'VAR_multiv_AR_^MERV_aic_residuals.pickle': '..\\results\\VAR_multiv_AR_^MERV_aic_residuals.pickle', 'VAR_multiv_AR_^MERV_bic_residuals.pickle': '..\\results\\VAR_multiv_AR_^MERV_bic_residuals.pickle', 'VAR_univ_AR_^MERV_aic_residuals.pickle': '..\\results\\VAR_univ_AR_^MERV_aic_residuals.pickle', 'VAR_univ_AR_^MERV_bic_residuals.pickle': '..\\results\\VAR_univ_AR_^MERV_bic_residuals.pickle'}


In [85]:
residual_df = pd.DataFrame()

for name, dir in all_residuals.items():
    dict_with_dfs = pd.read_pickle(dir)
    print(name)
    for stock in dict_with_dfs.keys():
        if type(dict_with_dfs[stock])==pd.Series: 
            # univariate models are saved as series
            df=pd.DataFrame(dict_with_dfs[stock])
        else:
            try:
                # multivariate models are saved as dataframes
                df=pd.DataFrame(dict_with_dfs[stock][f"{stock}_log_rets"])
            except: # TODO: SACAR ESTO! Es un chanchullo pq hay algunas que son guardadas como None
                pass

        modelname=name.replace("residuals.pickle", "").replace(params["tablename"], "").replace("__", "_").replace("__", "_")
        df.columns=[modelname+"_"+stock]
        residual_df = pd.merge(residual_df, df, left_index=True, right_index=True, how='outer')

GARCH_AR_^MERV_aic_residuals.pickle
GARCH_AR_^MERV_bic_residuals.pickle
HMM_univ_AR_^MERV_aic_residuals.pickle
HMM_univ_AR_^MERV_bic_residuals.pickle
VAR_multiv_AR_^MERV_aic_residuals.pickle
VAR_multiv_AR_^MERV_bic_residuals.pickle
VAR_univ_AR_^MERV_aic_residuals.pickle
VAR_univ_AR_^MERV_bic_residuals.pickle


In [86]:
residual_df.tail(4)

,GARCH_aic__^MERV,GARCH_aic__GGAL.BA,GARCH_aic__GGAL,GARCH_aic__YPFD.BA,GARCH_aic__YPF,GARCH_aic__EDN.BA,GARCH_aic__EDN,GARCH_aic__BMA.BA,GARCH_aic__BMA,GARCH_aic__BBAR.BA,GARCH_aic__BBAR,GARCH_aic__USD,GARCH_aic__USD_^MERV,GARCH_bic__^MERV,GARCH_bic__GGAL.BA,GARCH_bic__GGAL,GARCH_bic__YPFD.BA,GARCH_bic__YPF,GARCH_bic__EDN.BA,GARCH_bic__EDN,GARCH_bic__BMA.BA,GARCH_bic__BMA,GARCH_bic__BBAR.BA,GARCH_bic__BBAR,GARCH_bic__USD,GARCH_bic__USD_^MERV,HMM_univ_aic__^MERV,HMM_univ_aic__GGAL.BA,HMM_univ_aic__GGAL,HMM_univ_aic__YPFD.BA,HMM_univ_aic__YPF,HMM_univ_aic__EDN.BA,HMM_univ_aic__EDN,HMM_univ_aic__BMA.BA,HMM_univ_aic__BMA,HMM_univ_aic__BBAR.BA,HMM_univ_aic__BBAR,HMM_univ_bic__^MERV,HMM_univ_bic__GGAL.BA,HMM_univ_bic__GGAL,HMM_univ_bic__YPFD.BA,HMM_univ_bic__YPF,HMM_univ_bic__EDN.BA,HMM_univ_bic__EDN,HMM_univ_bic__BMA.BA,HMM_univ_bic__BMA,HMM_univ_bic__BBAR.BA,HMM_univ_bic__BBAR,VAR_multiv_aic__^MERV,VAR_multiv_aic__GGAL.BA,VAR_multiv_aic__GGAL,VAR_multiv_aic__YPFD.BA,VAR_multiv_aic__YPF,VAR_multiv_aic__EDN.BA,VAR_multiv_aic__EDN,VAR_multiv_aic__BMA.BA,VAR_multiv_aic__BMA,VAR_multiv_aic__BBAR.BA,VAR_multiv_aic__BBAR,VAR_multiv_bic__^MERV,VAR_multiv_bic__GGAL.BA,VAR_multiv_bic__GGAL,VAR_multiv_bic__YPFD.BA,VAR_multiv_bic__YPF,VAR_multiv_bic__EDN.BA,VAR_multiv_bic__EDN,VAR_multiv_bic__BMA.BA,VAR_multiv_bic__BMA,VAR_multiv_bic__BBAR.BA,VAR_multiv_bic__BBAR,VAR_univ_aic__^MERV,VAR_univ_aic__GGAL.BA,VAR_univ_aic__GGAL,VAR_univ_aic__YPFD.BA,VAR_univ_aic__YPF,VAR_univ_aic__EDN.BA,VAR_univ_aic__EDN,VAR_univ_aic__BMA.BA,VAR_univ_aic__BMA,VAR_univ_aic__BBAR.BA,VAR_univ_aic__BBAR,VAR_univ_bic__^MERV,VAR_univ_bic__GGAL.BA,VAR_univ_bic__GGAL,VAR_univ_bic__YPFD.BA,VAR_univ_bic__YPF,VAR_univ_bic__EDN.BA,VAR_univ_bic__EDN,VAR_univ_bic__BMA.BA,VAR_univ_bic__BMA,VAR_univ_bic__BBAR.BA,VAR_univ_bic__BBAR
2023-11-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.003217,0.045683,0.077352,0.035161,0.054353,0.041827,0.082862,0.104664,0.075407,0.051540,0.093391,-0.007429,0.061229,0.070109,0.052503,0.082948,0.083899,0.076541,0.104288,0.092949,0.072263,0.099728,-0.000800,0.058247,0.077120,0.042341,0.071404,0.039492,0.119300,0.160878,0.095132,0.167462,0.096655,0.025995,0.086179,0.082066,0.065960,0.109870,0.099103,0.116322,0.175330,0.120674,0.181001,0.106259
2023-11-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.012945,0.052759,0.064145,-0.051531,0.009840,-0.053755,0.012308,0.005081,0.044355,0.092173,0.066193,-0.096348,0.049665,0.068457,-0.079966,0.017590,-0.165180,0.000724,0.010261,0.051598,0.075671,0.058650,0.013490,0.029881,0.067031,-0.071825,0.005816,-0.074744,-0.013895,-0.007065,0.051028,-0.036962,0.067716,-0.098633,0.029069,0.076142,-0.130921,0.020882,-0.225791,-0.025054,-0.023719,0.050627,-0.023772,0.058986
2023-11-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.011718,-0.047031,-0.008868,-0.020326,-0.005301,0.001209,0.014970,-0.020637,-0.005075,-0.034970,-0.022179,-0.063684,-0.084285,-0.021771,-0.059538,-0.028366,-0.014113,0.012458,-0.091921,-0.006008,-0.026707,-0.059710,0.005626,-0.084261,-0.011169,-0.033593,-0.026315,-0.024719,-0.022455,-0.046048,-0.003551,-0.095770,-0.028878,-0.073576,-0.109025,-0.019512,-0.069612,-0.040667,-0.034736,-0.026902,-0.110806,-0.016833,-0.102010,-0.072732
2023-11-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.006020,0.004511,-0.005805,-0.006947,0.008136,-0.005428,0.030835,0.016358,0.004352,0.001771,0.009234,0.013443,0.000569,-0.008559,-

In [78]:
residual_df.isna().sum().sum()

126432

In [87]:
residual_df.isna().sum().sum()/(residual_df.shape[0]*residual_df.shape[1])*100
# wth

48.97579720474759